In [1]:
from IPython.display import display, HTML, Markdown
import json
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [3]:
id_label = [
    'continuous yellow',
    'continuous white',
    'dashed',
    'double dashed',
    "Botts' dots",
    'double continuous yellow',
]

In [4]:
# read /home/user/projects/llama/tusimple_gt/tusimple_lane_gt.json into gt_data
with open('/home/user/projects/llama/tusimple_gt/tusimple_lane_gt.json', 'r') as file:
    gt_data = json.load(file)

root_directory = '/home/user/projects/conditionallane/resa/data/tusimple/'
random_images_paths = [root_directory + gt_data[i]['raw_file'] for i in range(0, len(gt_data))]

In [5]:
# Load SBERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
yes_embedding = model.encode("yes")
no_embedding = model.encode("no")

def classify_answers(answers_list):
    # Calculate embeddings for answers
    answer_embeddings = model.encode(answers_list)

    # Calculate cosine similarity between each answer and "yes" and "no"
    similarities_yes = cosine_similarity(answer_embeddings, [yes_embedding])
    similarities_no = cosine_similarity(answer_embeddings, [no_embedding])

    # Determine the classification based on similarity
    classifications = [similarity_yes > similarity_no for similarity_yes, similarity_no in zip(similarities_yes, similarities_no)]
    
    # Extract boolean values from arrays
    classifications = [bool(classification[0]) for classification in classifications]
    
    return classifications

## Tusimple, lane marking

In [6]:
v = 'v1'
name = 'events'
res_paths = [f"{img.split('.')[0]}_{name}_blip_{v}.json" for img in random_images_paths]

In [7]:
print(res_paths[600])

/home/user/projects/conditionallane/resa/data/tusimple/clips/0313-1/40020/20_events_blip_v1.json


In [8]:
def create_blipv1_test_df(res_paths):
    answer_list = []
    for i, jpath in enumerate(res_paths):

        with open(jpath) as f:
            data = json.load(f)

        answers_list = data['events'][0]['answers']
        classifications = classify_answers(answers_list)
        classification_dict = dict(zip(id_label, classifications))
        classification_dict['image'] = random_images_paths[i]

        answer_list.append(classification_dict)


    return answer_list

In [9]:
with open(res_paths[500]) as f:
    data = json.load(f)

# Get the answers list
answers_list_1 = data['events'][0]['answers']

In [10]:
classify_answers(answers_list_1)

[False, False, False, False, False, False]

In [11]:
answer_list = create_blipv1_test_df(res_paths)

In [12]:
# convert list of json obj to a dataframe
df_v1_blip = pd.DataFrame(answer_list)
df_v1_blip.head()

,continuous yellow,continuous white,dashed,double dashed,Botts' dots,double continuous yellow,image
0,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
1,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
2,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
3,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
4,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...


In [13]:
# find rows with any True value in df_v1_blip
df_v1_blip[df_v1_blip.eq(True).any(1)]

/tmp/ipykernel_1501929/1125237395.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_v1_blip[df_v1_blip.eq(True).any(1)]


,continuous yellow,continuous white,dashed,double dashed,Botts' dots,double continuous yellow,image
13,False,True,False,False,False,False,/home/user/projects/conditionallane/resa/data...
37,False,True,True,False,False,False,/home/user/projects/conditionallane/resa/data...
81,False,False,True,False,False,False,/home/user/projects/conditionallane/resa/data...
82,True,True,True,False,False,False,/home/user/projects/conditionallane/resa/data...
85,False,True,True,False,False,False,/home/user/projects/conditionallane/resa/data...
...,...,...,...,...,...,...,...
3593,False,True,False,False,False,False,/home/user/projects/conditionallane/resa/data...
3594,False,False,True,False,False,False,/home/user/projects/conditionallane/resa/data...
3595,False,False,True,False,False,False,/home/user/projects/conditionallane/resa/data...
3606,False,True,False,False,False,False,/home/user/projects/conditionallane/resa/data...


In [14]:
len(df_v1_blip)

3626

In [21]:
# print just one of df_v1_blip image column in string format
print(df_v1_blip['image'][0])

/home/user/projects/conditionallane/resa/data/tusimple/clips/0313-1/6040/20.jpg


In [22]:
# read the test images in 
t_path = '/home/user/projects/datasets/prompt_datasets/TUSimple/test.json'
with open(t_path, 'r') as file:
    test_path = json.load(file)
# change the '/home/user/projects/datasets/prompt_datasets/TUSimple/train_set' to '/home/user/projects/conditionallane/resa/data/tusimple'
test_path = [x.replace('/home/user/projects/datasets/prompt_datasets/TUSimple/train_set', '/home/user/projects/conditionallane/resa/data/tusimple') for x in test_path]
test_path[0]

'/home/user/projects/conditionallane/resa/data/tusimple/clips/0313-1/34320/20.jpg'

In [23]:
# get subset of df_v1_blip that are in test_path
df_v1_blip_test = df_v1_blip[df_v1_blip['image'].isin(test_path)]
df_v1_blip_test

,continuous yellow,continuous white,dashed,double dashed,Botts' dots,double continuous yellow,image
1,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
2,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
6,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
8,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
10,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
...,...,...,...,...,...,...,...
3600,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
3610,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
3617,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...
3618,False,False,False,False,False,False,/home/user/projects/conditionallane/resa/data...


In [24]:
df_v1_blip_test.to_csv('./data/blip_lane_test.csv', index=False)

In [15]:
# save df to csv file
df_v1_blip.to_csv('./data/blip_lane.csv', index=False)

### Blip user

In [39]:
import os
from tqdm import tqdm


In [40]:
id_label = {
    0: 'bicycle',
    1: 'bus',
    2: 'car',
    3: 'motorcycle',
    4: 'pedestrian',
    5: 'rider',
    6: 'truck'
}

In [41]:
# Define paths
response_dir = '/home/user/projects/datasets/prompt_datasets/bdd100k/blip_response'
root_path = '/home/user/projects/datasets/prompt_datasets/bdd100k/images/'
json_file = '/home/user/projects/datasets/prompt_datasets/bdd100k/image_labels.json'

# Read the json file
with open(json_file, 'r') as f:
    data = json.load(f)

# For each image in data, join the path in root_path and the raw_file and save it in the new key fpath
for i, item in enumerate(data):
    file_path = item['raw_file']
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    data[i]['fpath'] = os.path.join(root_path, item['raw_file'])
    data[i]['prediction_path'] = os.path.join(response_dir, f'{file_name}.json')

data[0]

{'raw_file': '10k/train/00054602-3bf57337.jpg',
 'classes': [2],
 'fpath': '/home/user/projects/datasets/prompt_datasets/bdd100k/images/10k/train/00054602-3bf57337.jpg',
 'prediction_path': '/home/user/projects/datasets/prompt_datasets/bdd100k/blip_response/00054602-3bf57337.json'}

In [42]:
name = 'events'

# Process each image in the data object using its fpath and prediction_path
for i, item in enumerate(tqdm(data)):
    image_path = item['fpath']
    prediction_path = item['prediction_path']
    break

  0%|          | 0/2958 [00:00<?, ?it/s]


In [43]:
data[0]

{'raw_file': '10k/train/00054602-3bf57337.jpg',
 'classes': [2],
 'fpath': '/home/user/projects/datasets/prompt_datasets/bdd100k/images/10k/train/00054602-3bf57337.jpg',
 'prediction_path': '/home/user/projects/datasets/prompt_datasets/bdd100k/blip_response/00054602-3bf57337.json'}

In [44]:
with open(prediction_path) as f:
    test_data = json.load(f)

answers_list_1 = test_data['events'][0]['answers']
answers_list_1

['no', 'no', 'no', 'no', 'no', 'no', 'no']

In [45]:
classify_answers(answers_list_1)

[False, False, False, False, False, False, False]

In [46]:
def create_blipv1_test_df(label_data):
    id_label = {
        0: 'bicycle',
        1: 'bus',
        2: 'car',
        3: 'motorcycle',
        4: 'pedestrian',
        5: 'rider',
        6: 'truck'
    }
    answer_list = []
    # for i, jpath in enumerate(res_paths):
    for i, item in enumerate(tqdm(label_data, desc="Processing Ground Truth")):
        jpath = item['prediction_path']

        with open(jpath) as f:
            data = json.load(f)

        answers_list = data['events'][0]['answers']
        classifications = classify_answers(answers_list)
        classification_dict = dict(zip(id_label.values(), classifications))
        classification_dict['image'] = item['fpath']

        answer_list.append(classification_dict)


    return answer_list

In [47]:
answer_list = create_blipv1_test_df(data)

Processing Ground Truth: 100%|██████████| 2958/2958 [00:22<00:00, 133.71it/s]


In [48]:
answer_list[0]

{'bicycle': False,
 'bus': False,
 'car': False,
 'motorcycle': False,
 'pedestrian': False,
 'rider': False,
 'truck': False,
 'image': '/home/user/projects/datasets/prompt_datasets/bdd100k/images/10k/train/00054602-3bf57337.jpg'}

In [49]:
# convert list of json obj to a dataframe
df_v1_blip = pd.DataFrame(answer_list)
df_v1_blip.head()

,bicycle,bus,car,motorcycle,pedestrian,rider,truck,image
0,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
1,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
3,False,True,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
4,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...


In [50]:
df_v1_blip[df_v1_blip.eq(True).any(1)]

/tmp/ipykernel_1501929/1105450154.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_v1_blip[df_v1_blip.eq(True).any(1)]


,bicycle,bus,car,motorcycle,pedestrian,rider,truck,image
2,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
3,False,True,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
15,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
16,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
22,False,True,True,False,False,False,True,/home/user/projects/datasets/prompt_datasets/...
...,...,...,...,...,...,...,...,...
2940,False,False,True,False,False,False,True,/home/user/projects/datasets/prompt_datasets/...
2941,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2947,False,False,False,False,True,True,False,/home/user/projects/datasets/prompt_datasets/...
2948,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...


In [53]:
# read the test images in 
t_path = '/home/user/projects/datasets/prompt_datasets/bdd100k/test.json'
with open(t_path, 'r') as file:
    test_path = json.load(file)
test_path[0]

'/home/user/projects/datasets/prompt_datasets/bdd100k/images/10k/train/93b93cd9-394feaca.jpg'

In [54]:
df_v1_blip_test = df_v1_blip[df_v1_blip['image'].isin(test_path)]
df_v1_blip_test

,bicycle,bus,car,motorcycle,pedestrian,rider,truck,image
1,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
6,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
8,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
10,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
...,...,...,...,...,...,...,...,...
2934,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2935,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2945,False,False,False,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2951,False,False,False,False,False,True,False,/home/user/projects/datasets/prompt_datasets/...


In [55]:
df_v1_blip.to_csv('./data/blip_road_test.csv', index=False)

In [90]:
df_v1_blip.to_csv('./data/blip_road.csv', index=False)

# grand Truth lane marking

In [25]:
gt_data[0]

{'raw_file': 'clips/0313-1/6040/20.jpg',
 'classes': ['5', '5', '5', '5'],
 'label_counts': {'continuous yellow': 0,
  'continuous white': 0,
  'dashed': 0,
  'double dashed': 0,
  "Botts' dots": 4,
  'double continuous yellow': 0,
  'unknown': 0},
 'lane_gt': {'continuous yellow': False,
  'continuous white': False,
  'dashed': False,
  'double dashed': False,
  "Botts' dots": True,
  'double continuous yellow': False,
  'unknown': False}}

In [26]:
i = 10
gt_dict = gt_data[i]['lane_gt']
gt_dict['image'] = root_directory + gt_data[i]['raw_file']
gt_dict

{'continuous yellow': False,
 'continuous white': False,
 'dashed': False,
 'double dashed': False,
 "Botts' dots": True,
 'double continuous yellow': False,
 'unknown': False,
 'image': '/home/user/projects/conditionallane/resa/data/tusimple/clips/0313-1/7780/20.jpg'}

In [27]:
# enumrate over gt_data list of objects and create a new object like this: gt_dict = gt_data[i]['lane_gt'], gt_dict['image'] = root_directory + gt_data[i]['raw_file']
gt_list = []
for i in range(0, len(gt_data)):
    gt_dict = gt_data[i]['lane_gt']
    gt_dict['image'] = root_directory + gt_data[i]['raw_file']
    # in gt_dict remove 'unknown'
    if 'unknown' in gt_dict:
        del gt_dict['unknown']
    gt_list.append(gt_dict)

In [28]:
gt_list[0]

{'continuous yellow': False,
 'continuous white': False,
 'dashed': False,
 'double dashed': False,
 "Botts' dots": True,
 'double continuous yellow': False,
 'image': '/home/user/projects/conditionallane/resa/data/tusimple/clips/0313-1/6040/20.jpg'}

In [29]:
# list to dataframe
df_gt = pd.DataFrame(gt_list)
# save to csv file
df_gt.to_csv('./data/gt_lane.csv', index=False)

In [30]:
gt_test = df_gt[df_gt['image'].isin(test_path)]
gt_test

,continuous yellow,continuous white,dashed,double dashed,Botts' dots,double continuous yellow,image
1,False,False,False,False,True,False,/home/user/projects/conditionallane/resa/data...
2,True,False,True,False,False,True,/home/user/projects/conditionallane/resa/data...
6,True,False,False,False,True,False,/home/user/projects/conditionallane/resa/data...
8,True,False,False,False,True,False,/home/user/projects/conditionallane/resa/data...
10,False,False,False,False,True,False,/home/user/projects/conditionallane/resa/data...
...,...,...,...,...,...,...,...
3600,True,True,True,False,False,False,/home/user/projects/conditionallane/resa/data...
3610,True,False,True,True,False,False,/home/user/projects/conditionallane/resa/data...
3617,True,True,True,False,False,False,/home/user/projects/conditionallane/resa/data...
3618,True,True,True,False,False,False,/home/user/projects/conditionallane/resa/data...


In [31]:
gt_test.to_csv('./data/gt_lane_test.csv', index=False)

In [43]:
df_gt.head()

,continuous yellow,continuous white,dashed,double dashed,Botts' dots,double continuous yellow,image
0,False,False,False,False,True,False,/home/user/projects/conditionallane/resa/data...
1,False,False,False,False,True,False,/home/user/projects/conditionallane/resa/data...
2,True,False,True,False,False,True,/home/user/projects/conditionallane/resa/data...
3,False,True,False,False,True,False,/home/user/projects/conditionallane/resa/data...
4,False,True,True,False,False,False,/home/user/projects/conditionallane/resa/data...


## Ground Truth roadway user

In [56]:
data[0]

{'raw_file': '10k/train/00054602-3bf57337.jpg',
 'classes': [2],
 'fpath': '/home/user/projects/datasets/prompt_datasets/bdd100k/images/10k/train/00054602-3bf57337.jpg',
 'prediction_path': '/home/user/projects/datasets/prompt_datasets/bdd100k/blip_response/00054602-3bf57337.json'}

In [57]:
def create_blip_road_gt_df(label_data):
    id_label = {
        0: 'bicycle',
        1: 'bus',
        2: 'car',
        3: 'motorcycle',
        4: 'pedestrian',
        5: 'rider',
        6: 'truck'
    }
    answer_list = []
    for i, item in enumerate(tqdm(label_data, desc="Processing Ground Truth")):
        classes_list = item['classes']
        # create a dictionary with the classes list based on id_label if label is not exist in the classes list, set it to False
        gt_dict = dict(zip(id_label.values(), [False] * len(id_label)))
        for c in classes_list:
            gt_dict[id_label[c]] = True
        gt_dict['image'] = item['fpath']

        answer_list.append(gt_dict)


    return answer_list

In [58]:
gt_list = create_blip_road_gt_df(data)


Processing Ground Truth: 100%|██████████| 2958/2958 [00:00<00:00, 307113.01it/s]


In [59]:
gt_list[0]

{'bicycle': False,
 'bus': False,
 'car': True,
 'motorcycle': False,
 'pedestrian': False,
 'rider': False,
 'truck': False,
 'image': '/home/user/projects/datasets/prompt_datasets/bdd100k/images/10k/train/00054602-3bf57337.jpg'}

In [60]:
# list to dataframe
df_gt = pd.DataFrame(gt_list)
df_gt

,bicycle,bus,car,motorcycle,pedestrian,rider,truck,image
0,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
1,False,True,True,False,True,False,False,/home/user/projects/datasets/prompt_datasets/...
2,False,True,True,False,False,False,True,/home/user/projects/datasets/prompt_datasets/...
3,False,True,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
4,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
...,...,...,...,...,...,...,...,...
2953,False,False,True,False,False,False,True,/home/user/projects/datasets/prompt_datasets/...
2954,False,False,True,False,True,False,False,/home/user/projects/datasets/prompt_datasets/...
2955,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2956,False,True,True,True,False,True,False,/home/user/projects/datasets/prompt_datasets/...


In [61]:
df_gt_test = df_gt[df_gt['image'].isin(test_path)]
df_gt_test

,bicycle,bus,car,motorcycle,pedestrian,rider,truck,image
1,False,True,True,False,True,False,False,/home/user/projects/datasets/prompt_datasets/...
2,False,True,True,False,False,False,True,/home/user/projects/datasets/prompt_datasets/...
6,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
8,False,False,True,False,True,False,True,/home/user/projects/datasets/prompt_datasets/...
10,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
...,...,...,...,...,...,...,...,...
2934,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...
2935,False,False,True,False,True,False,True,/home/user/projects/datasets/prompt_datasets/...
2945,False,False,True,False,True,False,False,/home/user/projects/datasets/prompt_datasets/...
2951,False,False,True,False,False,False,False,/home/user/projects/datasets/prompt_datasets/...


In [62]:
df_gt_test.to_csv('./data/gt_user_test.csv', index=False)

In [ ]:
# save to csv file
df_gt.to_csv('./data/gt_user.csv', index=False)

## Acuracy

In [32]:
def align_dataframes(ground_truth_df, test_result_df):
    # Merge the dataframes based on the 'image' column
    merged_df = pd.merge(ground_truth_df, test_result_df, on='image', suffixes=('_ground_truth', '_test_result'), how='inner')

    # Drop rows with missing values after merge
    merged_df.dropna(inplace=True)

    # Reorder the rows to ensure the same order
    merged_df.sort_values(by='image', inplace=True)

    # Extract the common rows into separate dataframes
    common_ground_truth_df = merged_df[[col for col in merged_df.columns if col.endswith('_ground_truth')]]
    common_test_result_df = merged_df[[col for col in merged_df.columns if col.endswith('_test_result')]]

    # Rename columns to remove suffixes
    common_ground_truth_df.columns = [col.replace('_ground_truth', '') for col in common_ground_truth_df.columns]
    common_test_result_df.columns = [col.replace('_test_result', '') for col in common_test_result_df.columns]

    return common_ground_truth_df, common_test_result_df


In [33]:
def calculate_accuracy_precision_recall(ground_truth_df, test_result_df):
    metrics_dict = {}

    for column in ground_truth_df.columns:
        if column != 'image':
            # get the number of True values in the ground truth
            true_count = sum(ground_truth_df[column])
            # Get the ground truth and test result values for the current attribute column
            ground_truth_values = ground_truth_df[column]
            test_result_values = test_result_df[column]

            # Calculate TP, TN, FP, FN
            TP = sum((ground_truth_values == True) & (test_result_values == True))
            TN = sum((ground_truth_values == False) & (test_result_values == False))
            FP = sum((ground_truth_values == False) & (test_result_values == True))
            FN = sum((ground_truth_values == True) & (test_result_values == False))

            # Calculate accuracy, precision, recall, and F1 score
            accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
            precision = TP / (TP + FP) if (TP + FP) > 0 else 0
            recall = TP / (TP + FN) if (TP + FN) > 0 else 0

            # Calculate F1 score only if both precision and recall are not None

            if precision + recall > 0:
                f1 = 2 * (precision * recall) / (precision + recall)
            else:
                f1 = 0

            # Calculate AUROC and AUPRC
            if true_count > 0 and true_count < len(ground_truth_values):
                auroc = roc_auc_score(ground_truth_values, test_result_values)
                auprc = average_precision_score(ground_truth_values, test_result_values)
            else:
                auroc = 0
                auprc = 0

            # Store the metrics in a dictionary
            metrics_dict[column] = {
                'GT Count': true_count, 
                'Accuracy': accuracy, 
                'Precision': precision, 
                'Recall': recall, 
                'F1': f1, 
                'TP': TP, 
                'FP': FP, 
                'TN': TN, 
                'FN': FN,
                'AUROC': auroc,
                'AUPRC': auprc
            }

    # Convert metrics_dict to DataFrame
    metrics_df = pd.DataFrame.from_dict(metrics_dict, orient='index')
    # add average row to metrics_df
    metrics_df.loc['Average'] = metrics_df.mean()

    # Get the description of the DataFrame
    # description = metrics_df.describe()

    return metrics_df

In [69]:
# read df_gt from './data/gt_lane.csv'
df_gt = pd.read_csv('./data/gt_lane.csv')
# read df_v1_blip from ./data/blip_v1.csv
df_v1_blip = pd.read_csv('./data/blip_lane.csv')


In [34]:
# read df_gt from './data/gt_lane.csv'
df_gt = pd.read_csv('./data/gt_lane_test.csv')
# read df_v1_blip from ./data/blip_v1.csv
df_v1_blip = pd.read_csv('./data/blip_lane_test.csv')


In [35]:
col_list = ['AUROC', 'AUPRC', 'Accuracy', 'Precision', 'Recall', 'F1', 'TP', 'FP', 'TN',
       'FN']
gt_col = ['GT Count']
sum_metrics = ['Accuracy', 'Precision', 'AUROC', 'AUPRC', 'Recall']

### Lane marking

In [36]:
aligned_ground_truth_df, aligned_test_result_df = align_dataframes(df_gt, df_v1_blip)
metrics = calculate_accuracy_precision_recall(aligned_ground_truth_df, aligned_test_result_df)


In [37]:
# Assume metrics[gt_col] is your DataFrame
metrics[gt_col] = metrics[gt_col].fillna('').round(0)  # Fill NaN values with empty string and round to 0 decimal points
markdown_content = metrics[gt_col].to_markdown(floatfmt=".0f")
display(Markdown(markdown_content))

|                          |   GT Count |
|:-------------------------|-----------:|
| continuous yellow        |        416 |
| continuous white         |        519 |
| dashed                   |        567 |
| double dashed            |        117 |
| Botts' dots              |        519 |
| double continuous yellow |        165 |
| Average                  |        384 |

In [38]:
metrics[col_list].round(2)  

,AUROC,AUPRC,Accuracy,Precision,Recall,F1,TP,FP,TN,FN
continuous yellow,0.51,0.39,0.62,1.00,0.02,0.03,7.00,0.00,672.0,409.00
continuous white,0.50,0.48,0.52,0.50,0.02,0.04,11.00,11.00,558.0,508.00
dashed,0.51,0.53,0.49,0.67,0.03,0.05,16.00,8.00,513.0,551.00
double dashed,0.50,0.11,0.89,0.00,0.00,0.00,0.00,0.00,971.0,117.00
Botts' dots,0.50,0.48,0.52,0.00,0.00,0.00,0.00,0.00,569.0,519.00
double continuous yellow,0.50,0.15,0.85,0.00,0.00,0.00,0.00,0.00,923.0,165.00
Average,0.50,0.36,0.65,0.36,0.01,0.02,5.67,3.17,701.0,378.17


In [74]:
metrics_copy = metrics[col_list].copy().round(2)  
metrics_copy = metrics_copy.fillna('')  # Fill NaN values with empty string and round to 2 decimal points
markdown_content = metrics_copy.to_markdown()
display(Markdown(markdown_content))

|                          |   AUROC |   AUPRC |   Accuracy |   Precision |   Recall |   F1 |    TP |   FP |      TN |      FN |
|:-------------------------|--------:|--------:|-----------:|------------:|---------:|-----:|------:|-----:|--------:|--------:|
| continuous yellow        |    0.51 |    0.41 |       0.6  |        0.96 |     0.02 | 0.03 | 23    |    1 | 2169    | 1433    |
| continuous white         |    0.5  |    0.49 |       0.51 |        0.45 |     0.02 | 0.03 | 29    |   35 | 1803    | 1759    |
| dashed                   |    0.51 |    0.55 |       0.48 |        0.76 |     0.04 | 0.08 | 78    |   24 | 1656    | 1868    |
| double dashed            |    0.5  |    0.12 |       0.88 |        0    |     0    | 0    |  0    |    0 | 3195    |  431    |
| Botts' dots              |    0.5  |    0.47 |       0.53 |        0    |     0    | 0    |  0    |    0 | 1935    | 1691    |
| double continuous yellow |    0.5  |    0.14 |       0.86 |        0    |     0    | 0    |  0    |    0 | 3103    |  523    |
| Average                  |    0.5  |    0.36 |       0.64 |        0.36 |     0.01 | 0.02 | 21.67 |   10 | 2310.17 | 1284.17 |

### Roadway User

In [92]:
# read df_gt from './data/gt_lane.csv'
df_gt_road = pd.read_csv('./data/gt_user.csv')
# read df_v1_blip from ./data/blip_v1.csv
df_blip_road = pd.read_csv('./data/blip_road.csv')

In [63]:
# read df_gt from './data/gt_lane.csv'
df_gt_road = pd.read_csv('./data/gt_user_test.csv')
# read df_v1_blip from ./data/blip_v1.csv
df_blip_road = pd.read_csv('./data/blip_road_test.csv')

In [64]:
aligned_ground_truth_df, aligned_test_result_df = align_dataframes(df_gt_road, df_blip_road)


In [65]:
metrics = calculate_accuracy_precision_recall(aligned_ground_truth_df, aligned_test_result_df)


In [66]:
metrics[gt_col] = metrics[gt_col].fillna('').round(0)  # Fill NaN values with empty string and round to 0 decimal points
markdown_content = metrics[gt_col].to_markdown(floatfmt=".0f")
display(Markdown(markdown_content))

|            |   GT Count |
|:-----------|-----------:|
| bicycle    |         66 |
| bus        |        140 |
| car        |        883 |
| motorcycle |         54 |
| pedestrian |        346 |
| rider      |         66 |
| truck      |        258 |
| Average    |        259 |

In [67]:
metrics[col_list].round(2)  

,AUROC,AUPRC,Accuracy,Precision,Recall,F1,TP,FP,TN,FN
bicycle,0.51,0.09,0.93,0.67,0.03,0.06,2.00,1.00,821.00,64.00
bus,0.52,0.18,0.85,0.67,0.04,0.08,6.00,3.00,745.00,134.00
car,0.60,1.00,0.21,1.00,0.21,0.34,183.00,0.00,5.00,700.00
motorcycle,0.53,0.09,0.94,0.40,0.07,0.12,4.00,6.00,828.00,50.00
pedestrian,0.50,0.39,0.61,0.50,0.00,0.01,1.00,1.00,541.00,345.00
rider,0.53,0.08,0.90,0.18,0.09,0.12,6.00,28.00,794.00,60.00
truck,0.51,0.30,0.70,0.42,0.05,0.09,13.00,18.00,612.00,245.00
Average,0.53,0.30,0.73,0.55,0.07,0.12,30.71,8.14,620.86,228.29


In [68]:
metrics_copy = metrics[col_list].copy().round(2)  
metrics_copy = metrics_copy.fillna('')  # Fill NaN values with empty string and round to 2 decimal points
markdown_content = metrics_copy.to_markdown()
display(Markdown(markdown_content))

|            |   AUROC |   AUPRC |   Accuracy |   Precision |   Recall |   F1 |     TP |    FP |     TN |     FN |
|:-----------|--------:|--------:|-----------:|------------:|---------:|-----:|-------:|------:|-------:|-------:|
| bicycle    |    0.51 |    0.09 |       0.93 |        0.67 |     0.03 | 0.06 |   2    |  1    | 821    |  64    |
| bus        |    0.52 |    0.18 |       0.85 |        0.67 |     0.04 | 0.08 |   6    |  3    | 745    | 134    |
| car        |    0.6  |    1    |       0.21 |        1    |     0.21 | 0.34 | 183    |  0    |   5    | 700    |
| motorcycle |    0.53 |    0.09 |       0.94 |        0.4  |     0.07 | 0.12 |   4    |  6    | 828    |  50    |
| pedestrian |    0.5  |    0.39 |       0.61 |        0.5  |     0    | 0.01 |   1    |  1    | 541    | 345    |
| rider      |    0.53 |    0.08 |       0.9  |        0.18 |     0.09 | 0.12 |   6    | 28    | 794    |  60    |
| truck      |    0.51 |    0.3  |       0.7  |        0.42 |     0.05 | 0.09 |  13    | 18    | 612    | 245    |
| Average    |    0.53 |    0.3  |       0.73 |        0.55 |     0.07 | 0.12 |  30.71 |  8.14 | 620.86 | 228.29 |